# Summary 
Generates the geojson hexes+dissolved cities file.

Demonstrates calculation of 50/10 access levels from the NBD map data against arbitrary 
geometry other than the precomputed hexagons.

Calculating this requires joining the geometry from the Canada wide hexagons geometry 
with the tabular `PHH_Speeds_Current-PHH_Vitesses_Actuelles.csv` dataset. 

Loading the data and executing the spatial joins takes a total of 7 minutes.

In [1]:
import sys
sys.path.append("..")

%load_ext autoreload
%autoreload 1
%aimport src.datasets.joins
%aimport src.datasets.loading.statcan

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from src.datasets.loading import statcan
from src.datasets.loading import ookla
from src.datasets import overlays

from src.datasets import joins
import src

In [3]:
popctrs = statcan.boundary('population_centres')

In [4]:
popctrs.loc[lambda s:s.PCNAME == "Lloydminster"]

,PCUID,PCNAME,PCTYPE,PCPUID,PCCLASS,PRUID,PRNAME,CMAUID,CMANAME,CMATYPE,CMAPUID,geometry
657,0478,Lloydminster,1,480478,3,48,Alberta,840,Lloydminster (Alberta part / partie de l'Alberta),D,48840,"POLYGON ((5024336.049 2072898.043, 5024239.789..."
774,0478,Lloydminster,1,470478,3,47,Saskatchewan,840,Lloydminster (Saskatchewan part / partie de la...,D,47840,"POLYGON ((5024135.374 2071102.100, 5024109.260..."


In [5]:
custom_areas = joins.hexagons_small_popctrs_combined()
o = custom_areas

/home/jovyan/src/datasets/overlays.py:25: UserWarning: `keep_geom_type=True` in overlay resulted in 403 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  ol = gp.overlay(left, right, how="union")
/opt/conda/lib/python3.10/site-packages/geopandas/geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [6]:
phh_speed_csv = src.config.DATA_DIRECTORY / 'PHH' / 'CRTC_Speed_Data' / 'PHH_Speeds_Current-PHH_Vitesses_Actuelles.csv'
phh_speeds = pd.read_csv(phh_speed_csv)

phh_geo = statcan.phh_geometry()

phh = phh_geo.merge(phh_speeds, on='PHH_ID')
phh = phh.merge(statcan.phh_data(), on='PHH_ID')
phh = phh.loc[lambda s:s.Type !=8] # drop PHH nulls

In [7]:
from src.config import DATA_DIRECTORY
PHH_DIR = DATA_DIRECTORY / "PHH"
list(PHH_DIR.glob("./PHH_Data_MapInfo/*.TAB"))

[]

In [ ]:
o = joins.add_phh_pop(custom_areas, phh, 'HEXUID_PCPUID')

In [ ]:
del o['HEXuid_HEXidu'] # we don't want these, they're quite long in large cities

In [ ]:
popctrs = statcan.boundary('population_centres') # use this to generate a filtering region around cities

In [ ]:
#Can't plot the whole country, too many hexagons.
xmin, ymin, xmax, ymax = popctrs.loc[lambda s:s.PCNAME=="Lloydminster"].buffer(100000).total_bounds
o.cx[xmin:xmax,ymin:ymax].loc[lambda s:(s['Pop2016'] > 0) | (s['TDwell2016_TLog2016'] > 0) | (s['URDwell2016_RH2016']>0)].explore('Pop_Avail_50_10',  scheme='equalinterval', k = 4,)

In [ ]:
zeroable_cols = [
    'Pop2016', 'TDwell2016_TLog2016', 'URDwell2016_RH2016',
    'Pop2016_at_50_10_Combined', 'TDwell2016_at_50_10_Combined',
    'URDwell_at_50_10_Combined'
       ]
o[zeroable_cols] = o[zeroable_cols].fillna(0)

In [ ]:
o

In [ ]:
from pathlib import Path
from src import config

In [ ]:
derived_geometry = (Path(src.config.DATA_DIRECTORY) / 'processed' / 'geometries').resolve()
derived_geometry.mkdir(exist_ok=True, parents=True)

o.to_crs("EPSG:4326").to_file(derived_geometry / 'hexagons_w_dissolved_smaller_popctrs.geojson', driver='GeoJSON')

In [ ]:
o.to_file(derived_geometry / 'hexagons_w_dissolved_smaller_popctrs', driver="MapInfo File")